In [1]:
!python3 --version

Python 3.9.19


In [2]:
! pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.2 MB/s eta 0:00:00 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 9.9 MB/s eta 0:00:0010.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 9.5 MB/s eta 0:00:009.3 MB/s eta 0:00:01
  Created wheel for peewee: filename=peewee-3.17.8-cp39-cp39-linux_x86_64.whl size=311594 sha256=9427b166032bdca0d0b8b09b15e0735d7b6763341ace90d3e4454088fab8e268
  Stored in directory: /home/hadoop/.cache/pip/wheels/06/b3/7f/ed42a7c83ad89f578928833f5789212c694a015b8bd6a407a1
Successfully built peewee


In [3]:
import yfinance as yf
data = yf.download("AAPL IBM", start="2009-01-01", end="2019-12-31")

[*********************100%***********************]  2 of 2 completed


In [4]:
data['Open']

Ticker,AAPL,IBM
Date,,
2009-01-02,3.067143,80.200768
2009-01-05,3.327500,82.619499
2009-01-06,3.426786,83.279160
2009-01-07,3.278929,83.967499
2009-01-08,3.229643,83.948372
...,...,...
2019-12-23,70.132500,129.808792
2019-12-24,71.172501,129.646271
2019-12-26,71.205002,129.043976


In [5]:
%%bash
$HADOOP_HOME/sbin/start-dfs.sh
$HADOOP_HOME/sbin/start-yarn.sh

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [nmit]
Starting resourcemanager
Starting nodemanagers


In [6]:
!jps

23825 NodeManager
22758 DataNode
23414 ResourceManager
24118 Jps
23065 SecondaryNameNode
22494 NameNode


In [16]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='localhost', port=9000)
with hdfs.open('/home/hadoop/AAPL_IBM_open.csv', 'wb') as f:
    data['Open'].to_csv(f,header=True)

2024-12-16 10:35:13.592283, p21513, th128498892310016, WARNING the number of nodes in pipeline is 1 [nmit(127.0.0.1)], is less than the expected number of replica 3 for block [block pool ID: BP-835869387-127.0.1.1-1727256748010 block ID 1073742159_1335] file /home/hadoop/AAPL_IBM_open.csv


In [17]:
hdfs.ls('/home/hadoop/.')

['/home/hadoop/AAPL_IBM_open.csv', '/home/hadoop/user']

In [20]:
%%file stock_analysis.py

from mrjob.job import MRJob

import re
import sys

class StockAnalysis(MRJob):

   def mapper(self, key, value):
      date, apple_open, samsung_open = value.split(',')
      #print(value, file=sys.stderr)
      year = date[:4]
      month = date[5:7]
      if (month=='10' or month=='11' or month=='12'):
         apple_key = 'apple_%s' % year
         samsung_key = 'samsung_%s' % year
         yield(apple_key, float(apple_open))
         yield(samsung_key, float(samsung_open))
      
   def reducer(self, key, values):
      yield(key, max(values))

if __name__ == '__main__':
   StockAnalysis.run()

Overwriting stock_analysis.py


In [11]:
!python3 /home/hadoop/stock_analysis.py -r hadoop hdfs://AAPL_IBM_open.csv

Traceback (most recent call last):
  File "/home/hadoop/stock_analysis.py", line 2, in <module>
    from mrjob.job import MRJob
ModuleNotFoundError: No module named 'mrjob'


In [12]:
!pip install mrjob

In [13]:
!python3 /home/hadoop/stock_analysis.py -r hadoop hdfs://AAPL_IBM_open.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/hadoop/hadoop-3.4.0/bin...
Found hadoop binary: /home/hadoop/hadoop-3.4.0/bin/hadoop
STDERR: 2024-12-16 10:30:38,963 WARN fs.FileSystem: Failed to initialize filesystem hdfs://AAPL_IBM_open.csv: java.io.IOException: Incomplete HDFS URI, no host: hdfs://AAPL_IBM_open.csv
STDERR: ls: Incomplete HDFS URI, no host: hdfs://AAPL_IBM_open.csv
Traceback (most recent call last):
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/fs/hadoop.py", line 307, in exists
    return_code = self.invoke_hadoop(
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/fs/hadoop.py", line 183, in invoke_hadoop
    raise CalledProcessError(proc.returncode, args)
subprocess.CalledProcessError: Command '['/home/hadoop/hadoop-3.4.0/bin/hadoop', 'fs', '-ls', 'hdfs://AAPL_IBM_open.csv']' returned non-zero exit status 1.

During handling of the above

In [14]:
!python3 /home/hadoop/stock_analysis.py -r hadoop hdfs:/AAPL_IBM_open.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Traceback (most recent call last):
  File "/home/hadoop/stock_analysis.py", line 24, in <module>
    StockAnalysis.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1146, in _check_input_path
    raise IOError(
O

In [15]:
!python3 /home/hadoop/stock_analysis.py -r hadoop hdfs:/home/hadoop/AAPL_IBM_open.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Traceback (most recent call last):
  File "/home/hadoop/stock_analysis.py", line 24, in <module>
    StockAnalysis.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1146, in _check_input_path
    raise IOError(
O

In [18]:
!python3 /home/hadoop/stock_analysis.py -r hadoop hdfs:/home/hadoop/user/AAPL_IBM_open.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Traceback (most recent call last):
  File "/home/hadoop/stock_analysis.py", line 24, in <module>
    StockAnalysis.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1146, in _check_input_path
    raise IOError(
O

In [22]:
!python3 stock_analysis.py -r hadoop hdfs:/home/hadoop/APL_IBM_open.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Traceback (most recent call last):
  File "/home/hadoop/stock_analysis.py", line 24, in <module>
    StockAnalysis.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/home/nmit/anaconda3/envs/stock/lib/python3.9/site-packages/mrjob/runner.py", line 1146, in _check_input_path
    raise IOError(
O

In [23]:
!python3 /home/hadoop/stock_analysis.py -r hadoop hdfs:///home/hadoop/AAPL_IBM_open.csv

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/hadoop/hadoop-3.4.0/bin...
Found hadoop binary: /home/hadoop/hadoop-3.4.0/bin/hadoop
Using Hadoop version 3.4.0
Looking for Hadoop streaming jar in /home/hadoop/hadoop-3.4.0...
Found Hadoop streaming jar: /home/hadoop/hadoop-3.4.0/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar
Creating temp directory /tmp/stock_analysis.hadoop.20241216.051012.318261
uploading working dir files to hdfs:///user/hadoop/tmp/mrjob/stock_analysis.hadoop.20241216.051012.318261/files/wd...
Copying other local files to hdfs:///user/hadoop/tmp/mrjob/stock_analysis.hadoop.20241216.051012.318261/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar1775918782296022663/] [] /tmp/streamjob2970879527946789476.jar tmpDir=null
  Connecting to ResourceManager at /127.0.0.1:8032
  Connecting to ResourceManager at /127.0.0.1:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/